In [2]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("../data/labelled/15.csv")
#survey = pd.read_csv("../data/raw/surveyresult.csv")

In [5]:
data.head()

,session,timestamp,temp_mean,temp_std,temp_min,temp_max,temp_range,acc_magnitude_mean,acc_magnitude_std,acc_magnitude_max,...,hr_std,hr_min,hr_max,hr_range,hr_acc_ratio,hrv_mean_ibi,hrv_rmssd,hrv_sdnn,hrv_pnn50,stress_level
0,15_1594140175,2020-07-07 16:42:56,30.1200,0.010160,30.11,30.13,0.02,1.015027,0.028650,1.153396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,15_1594140175,2020-07-07 16:43:04,30.3775,0.206210,30.13,30.67,0.54,1.033760,0.171736,1.731557,...,2.133017,81.00,87.25,6.25,1.254284,NaN,NaN,NaN,NaN,0
2,15_1594140175,2020-07-07 16:43:12,30.6400,0.061801,30.55,30.73,0.18,1.026084,0.108528,1.923208,...,0.759218,78.67,80.75,2.08,1.212266,NaN,NaN,NaN,NaN,0
3,15_1594140175,2020-07-07 16:43:20,30.5050,0.024363,30.45,30.53,0.08,1.014604,0.048561,1.419211,...,1.720837,73.57,78.25,4.68,1.170831,NaN,NaN,NaN,NaN,0
4,15_1594140175,2020-07-07 16:43:28,30.7225,0.095275,30.55,30.81,0.26,1.019964,0.090604,1.456649,...,0.986226,70.38,72.92,2.54,1.090897,NaN,NaN,NaN,NaN,0


In [12]:
(data["stress_level"] == 1).sum()

np.int64(0)

In [10]:
data.shape

(32814, 46)

In [5]:
data.columns

Index(['session', 'timestamp', 'hr_mean', 'hr_std', 'hr_min', 'hr_max',
       'hr_range', 'temp_mean', 'temp_std', 'temp_min', 'temp_max',
       'temp_range', 'acc_magnitude_mean', 'acc_magnitude_std',
       'acc_magnitude_max', 'acc_magnitude_min', 'acc_activity_level',
       'eda_mean', 'eda_std', 'eda_max', 'eda_min', 'eda_range', 'eda_slope',
       'num_scr_peaks', 'mean_scr_amplitude', 'mean_scr_peak_width',
       'mean_scr_area', 'bvp_mean', 'bvp_std', 'bvp_max', 'bvp_min',
       'bvp_range', 'bvp_systolic_amp_mean', 'bvp_systolic_amp_std',
       'bvp_pulse_width_mean', 'bvp_pulse_width_std', 'bvp_rise_time_mean',
       'bvp_rise_time_std', 'bvp_pulse_rate', 'eda_acc_ratio', 'hr_acc_ratio',
       'hrv_mean_ibi', 'hrv_rmssd', 'hrv_sdnn', 'hrv_pnn50'],
      dtype='object')

In [6]:
survey.columns

Index(['ID', 'Start time', 'End time', 'duration', 'date', 'Stress level',
       'COVID related', 'Treating a covid patient', 'Patient in Crisis',
       'Patient or patient's family', 'Doctors or colleagues',
       'Administration, lab, pharmacy, radiology, or other ancilliary services\n',
       'Increased Workload', 'Technology related stress', 'Lack of supplies',
       'Documentation', 'Competency related stress',
       'Saftey (physical or physiological threats)',
       'Work Environment - Physical or others: work processes or procedures',
       'Description'],
      dtype='object')